In [ ]:
import tkinter as tk
from tkinter import ttk
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import requests
import random

# Replace with your Alpha Vantage API key
ALPHA_VANTAGE_API_KEY = 'YZZSR98L7VIU0KM2'

# Initialize the root window
root = tk.Tk()
root.title("TradeSTREAM")
root.configure(bg="navy")
root.geometry("1200x800")  # Increased window size for better graph visibility

# Adding Project Details (heading and subheading)
project_header = tk.Label(root, text="TradeSTREAM", font=("Arial", 20, "bold"), fg="yellow", bg="navy")
project_header.pack(pady=10)

sub_header = tk.Label(root, text="Predicting the Future of Stock Market", font=("Arial", 14), fg="white", bg="navy")
sub_header.pack(pady=5)

# Team Details
team_label = tk.Label(root, text="Our Team: AYUSH KUMAR, PRANAV RANA, SARTHAK RAGHAV, MAYANK RAJ", font=("Arial", 12), fg="orange", bg="navy")
team_label.pack(pady=5)

# Date and Time Display
def update_time():
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    time_label.config(text=f"Current Time: {current_time}")
    root.after(1000, update_time)

time_label = tk.Label(root, text="", font=("Arial", 12), fg="white", bg="navy")
time_label.pack(pady=5)
update_time()

# Investment Tracking (Dictionary to track user's portfolio)
portfolio = {}

# Dummy stock data (Placeholder for API integration)
genres = ["Technology", "Finance", "Healthcare", "Consumer Goods", "Energy", "Industrials"]

# Function to get stock price using Alpha Vantage API
def get_stock_price(symbol):
    url = f"https://www.alphavantage.co/query"
    params = {
        'function': 'TIME_SERIES_INTRADAY',
        'symbol': symbol,
        'interval': '5min',  # You can change the interval if needed (e.g., '1min', '15min', '30min', etc.)
        'apikey': ALPHA_VANTAGE_API_KEY
    }
    
    try:
        response = requests.get(url, params=params)
        data = response.json()
        
        # Extracting the latest stock price from the response
        time_series = data.get('Time Series (5min)', {})
        if time_series:
            latest_time = sorted(time_series.keys())[0]  # Get the latest timestamp
            latest_data = time_series[latest_time]
            price = float(latest_data['4. close'])  # Close price is the most recent trading price
            return price
        else:
            print("Error: No data available.")
            return None
    except Exception as e:
        print(f"Error fetching data from API: {e}")
        return None

# Function to update stock data in dropdown
def update_stocks(event=None):
    selected_genre = genre_var.get()
    stock_menu["values"] = stock_data[selected_genre]
    stock_menu.set(stock_data[selected_genre][0])

# Function to predict stock movement (Simulating for now)
def predict():
    stock = stock_var.get()
    genre = genre_var.get()
    if stock and genre:
        # Fetch real stock data from API
        price = get_stock_price(stock)
        if price is not None:
            stock_info_label.config(text=f"Current Price for {stock}: ₹{price:.2f}")
            plot_graph(price)
        else:
            stock_info_label.config(text="Error fetching stock data.")
        # Simulating price movement prediction
        predicted_change = random.choice(["Price is predicted to go up!", "Price is predicted to go down!"])
        stock_info_label.config(text=f"Stock: {stock} ({predicted_change})")
        plot_graph(price)

def plot_graph(current_price):
    # Create a simple graph showing price movement
    dates = ["Today", "1 Week Ago", "1 Month Ago"]
    prices = [current_price, current_price * 0.98, current_price * 1.02]  # Simulating price change
    
    plt.figure(figsize=(6, 3))  # Ensure graph is big enough
    plt.plot(dates, prices, marker='o')
    plt.title(f"Price Movement for Selected Stock")
    plt.ylabel('Price (₹)')
    plt.xlabel('Time')
    
    canvas = FigureCanvasTkAgg(plt.gcf(), master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

# UI elements for genre selection
genre_var = tk.StringVar()
genre_label = tk.Label(root, text="Select Stock Genre", font=("Arial", 12), fg="white", bg="navy")
genre_label.pack(pady=20)
genre_menu = ttk.Combobox(root, textvariable=genre_var, values=genres, state="readonly")
genre_menu.pack(pady=10)
genre_menu.bind("<<ComboboxSelected>>", update_stocks)

# Stock data dictionary (Placeholder for actual stock list)
stock_data = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA"],
    "Finance": ["JPM", "GS", "C", "BAC", "WFC", "MS"],
    "Healthcare": ["PFE", "JNJ", "MRK", "ABBV", "BMY", "CVS"],
    "Consumer Goods": ["KO", "PEP", "PG", "MCD", "SBUX", "TGT"],
    "Energy": ["XOM", "CVX", "COP", "OXY", "EOG", "SLB"],
    "Industrials": ["GE", "BA", "CAT", "DE", "LMT", "RTX"]
}

# UI elements for stock selection
stock_var = tk.StringVar()
stock_label = tk.Label(root, text="Select Stock", font=("Arial", 12), fg="white", bg="navy")
stock_label.pack(pady=20)
stock_menu = ttk.Combobox(root, textvariable=stock_var, state="readonly")
stock_menu.pack(pady=10)

# Button to predict stock performance
predict_button = tk.Button(root, text="Predict", font=("Arial", 12), bg="orange", command=predict)
predict_button.pack(pady=20)

# Frame for displaying stock graphs (Aligned to the right)
graph_frame = tk.Frame(root)
graph_frame.pack(side=tk.RIGHT, padx=20, pady=20, fill=tk.Y)

# Frame for displaying stock information (Aligned to the left)
info_frame = tk.Frame(root, bg="navy")
info_frame.pack(side=tk.LEFT, padx=20, pady=20, fill=tk.Y)

# Stock information label
stock_info_label = tk.Label(info_frame, text="", font=("Arial", 12), fg="white", bg="navy")
stock_info_label.pack(pady=20)

# Portfolio Tracking (Display user's investment portfolio)
portfolio_label = tk.Label(info_frame, text="Your Investment Portfolio", font=("Arial", 14), fg="yellow", bg="navy")
portfolio_label.pack(pady=20)

# Display the portfolio in a Label
portfolio_display_label = tk.Label(info_frame, text="No stocks added yet.", font=("Arial", 12), fg="white", bg="navy")
portfolio_display_label.pack(pady=5)

def add_to_portfolio():
    stock = stock_var.get()
    if stock not in portfolio:
        portfolio[stock] = 0
    portfolio[stock] += 1  # Increment stock by 1 in portfolio
    portfolio_display_label.config(text=f"Portfolio: {portfolio}")

# Button to add stocks to portfolio
add_button = tk.Button(info_frame, text="Add to Portfolio", font=("Arial", 12), bg="green", command=add_to_portfolio)
add_button.pack(pady=20)

# Function for investment simulation
def invest():
    # Get investment inputs
    invest_amount = float(invest_amount_entry.get())  # Investment amount
    invest_duration = int(invest_duration_entry.get())  # Duration in months
    
    stock = stock_var.get()
    if stock and invest_amount and invest_duration:
        # Fetch stock price using Alpha Vantage API
        price = get_stock_price(stock)
        if price is not None:
            projected_prices = [price]  # Initial price
            for month in range(1, invest_duration+1):
                projected_prices.append(projected_prices[-1] * (1 + random.uniform(-0.05, 0.05)))  # Simulating growth/decline
            profit_or_loss = projected_prices[-1] - invest_amount
            investment_result_label.config(text=f"Final Investment Value: ₹{projected_prices[-1]:.2f}\nProfit/Loss: ₹{profit_or_loss:.2f}")
            plot_investment_graph(projected_prices)

# Function to plot investment result over time
def plot_investment_graph(projected_prices):
    months = list(range(1, len(projected_prices)+1))
    plt.figure(figsize=(6, 3))
    plt.plot(months, projected_prices, marker='o', color='blue')
    plt.title(f"Investment Growth over Time")
    plt.ylabel('Value (₹)')
    plt.xlabel('Months')
    
    canvas = FigureCanvasTkAgg(plt.gcf(), master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

# Inputs for investment amount and duration
invest_amount_label = tk.Label(info_frame, text="Enter Investment Amount (₹):", font=("Arial", 12), fg="white", bg="navy")
invest_amount_label.pack(pady=10)
invest_amount_entry = tk.Entry(info_frame, font=("Arial", 12), fg="black")
invest_amount_entry.pack(pady=5)

invest_duration_label = tk.Label(info_frame, text="Enter Duration (Months):", font=("Arial", 12), fg="white", bg="navy")
invest_duration_label.pack(pady=10)
invest_duration_entry = tk.Entry(info_frame, font=("Arial", 12), fg="black")
invest_duration_entry.pack(pady=5)

# Button to start investment simulation
invest_button = tk.Button(info_frame, text="Invest", font=("Arial", 12), bg="green", command=invest)
invest_button.pack(pady=20)

# Investment result display
investment_result_label = tk.Label(info_frame, text="", font=("Arial", 12), fg="white", bg="navy")
investment_result_label.pack(pady=20)

root.mainloop()


# Assistant
The error you're encountering is a `FileNotFoundError`, which means that the code is trying to access a file that does not exist at the specified path. In this case, it seems to be looking for an image file located at `'C:\\Users\\AA\\EightcapLabs_blue_fundamentals_1320761489-1024x683.jpg'`, but it cannot find it.

To fix this error, you should:
1. Ensure that the file exists at the specified path.
2. If the file is located in a different directory, update the path in your code to point to the correct location.
3. Alternatively, you can use a relative path if the image is in the same directory as your script.

Would you like me to provide an example of how to check for the file and handle the error gracefully?